### **RunnableLambda**

- 사용자 정의 함수 실행 기능
- 데이터 전처리, 계산, 외부 API와 상호 작용을 `RunnableLambda`로 사용 할 수 있음
- 사용자 정의 함수가 받을 수 있는 인자는 `1개`


In [ ]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [27]:
def length_function(text):  # 텍스트의 길이를 반환하는 함수
    return len(text)


def _multiple_length_function(text1, text2):  # 두 텍스트의 길이를 곱하는 함수
    return len(text1) * len(text2)

# 2개 인자를 받는 함수로 연결하는 wrapper 함수
def multiple_length_function( _dict,):  # 딕셔너리에서 "text1"과 "text2"의 길이를 곱하는 함수
    return _multiple_length_function(_dict["text1"], _dict["text2"])




prompt = ChatPromptTemplate.from_template("what is {a} + {b}? answer the only number")

llm = ChatOpenAI()

chain1 = prompt | llm

chain = (
    {
        "a": itemgetter("input_1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("input_1"), "text2": itemgetter("input_2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"input_1": "bar", "input_2": "gah"})

'12'